<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/Collaborative_filltering_appliances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 구매 기록이 한번도 없는 상품은 추천 목록에서 제외 -> 희소문제를 해결하기 위함 
  - 전자 제품 군의 14,950개의 상품 중에서 한번도 팔리지 않은 상품은 약 10,000개로 분석에 포함될 시 희소문제를 일으킬 가능성이 높음

- 편향 보정을 위해 스마트폰을 구매한 일부 비율을 뺄까?

## 구매 횟수 필터 없는 추천 시스템

In [3]:
from pandas.io import gbq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
#pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.simplefilter("ignore")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [36]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'appliances%' AND event_type = 'purchase'
GROUP BY user_id, product_id
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [37]:
df_ratings = df

In [71]:
base = df_ratings['purchase_count'].mean()
base

1.1954411413086794

In [4]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'appliances%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [5]:
product_data = df_product
product_data.head()

,product_id,brand,category_code
0,8700226,babyliss,appliances.personal.hair_cutter
1,5000356,janome,appliances.sewing_machine
2,2900936,galanz,appliances.kitchen.microwave
3,5000475,janome,appliances.sewing_machine
4,3100825,scarlett,appliances.kitchen.blender


In [42]:
df_fin = pd.pivot_table(df, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')

In [43]:
df_fin

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
384989212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429474454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433754231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440756116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450756413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566221288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566227497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566235555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_base_line = df_fin.fillna(base)

In [46]:
df_final = df_fin.fillna(0)

In [47]:
df_final['total'] = df_final.sum(axis=1)
df_filtered = df_final.loc[df_final['total'] > 1]
df_filtered.drop(columns='total', inplace = True)

df_filtered

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481341715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494312717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500246310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566094079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566122364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566144303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
matrix = df_filtered.values
user_ratings_mean = np.mean(matrix, axis = 1)

In [49]:
user_ratings_mean_reshape = user_ratings_mean.reshape(-1, 1)

In [50]:
matrix_user_mean = matrix - user_ratings_mean_reshape

In [51]:
pd.DataFrame(matrix_user_mean, columns = df_filtered.columns, index = df_filtered.index).head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,...,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240
481341715,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
494312717,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
500246310,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
512363712,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413


In [52]:
from scipy.sparse.linalg import svds

In [102]:
U, sigma, Vt = svds(matrix_user_mean, k = 200)

In [103]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(12561, 200)
(200,)
(200, 4838)


In [104]:
sigma = np.diag(sigma)

In [105]:
sigma.shape

(200, 200)

In [106]:
sigma[0]

array([9.32635077, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [107]:
dot_matrix = np.dot(U, sigma)

In [108]:
df_svd_predicted_purchase = np.dot(dot_matrix, Vt)

In [109]:
df_svd_predicted_purchase = df_svd_predicted_purchase + user_ratings_mean_reshape

In [110]:
df_svd_preds = pd.DataFrame(df_svd_predicted_purchase, columns = df_filtered.columns, index = df_filtered.index)
df_svd_preds.head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.000391,0.000394,0.000394,0.000380,0.000801,0.000394,0.000378,0.000389,0.000396,0.000440,...,0.000382,0.000394,0.001167,-0.002678,0.000394,0.000389,0.000394,0.000394,0.000394,0.000396
481341715,0.000198,0.000199,0.000199,0.000200,0.000103,0.000199,0.000195,0.000191,0.000205,0.000197,...,0.000196,0.000199,-0.000089,0.000062,0.000199,0.000197,0.000199,0.000199,0.000199,0.000191
494312717,-0.000121,-0.000114,-0.000114,-0.000115,-0.002820,-0.000114,-0.000129,-0.000115,-0.000143,-0.000133,...,-0.000133,-0.000114,0.000070,-0.000059,-0.000114,-0.000117,-0.000114,-0.000114,-0.000114,-0.000175
500246310,-0.000094,-0.000094,-0.000094,-0.000096,-0.001699,-0.000094,-0.000095,-0.000092,-0.000116,-0.000086,...,-0.000103,-0.000094,0.000027,-0.000017,-0.000094,-0.000095,-0.000094,-0.000094,-0.000094,-0.000072
512363712,-0.000245,-0.000228,-0.000228,-0.000252,-0.000253,-0.000228,-0.000272,-0.000268,-0.000275,-0.000429,...,-0.000260,-0.000228,0.000033,0.000024,-0.000228,-0.000246,-0.000228,-0.000228,-0.000228,-0.000298


In [6]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['product_id'].isin(user_history['product_id'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [111]:
already_rated, predictions = recommend_movies(df_svd_preds, 566235555, product_data, df_ratings, 10)

In [112]:
predictions

,product_id,brand,category_code,Predictions
937,2501751,artel,appliances.kitchen.oven,0.261552
247,2500859,asel,appliances.kitchen.oven,0.057857
447,2900561,None,appliances.kitchen.microwave,0.055849
446,2900561,midea,appliances.kitchen.microwave,0.055849
1774,3601551,midea,appliances.kitchen.washer,0.032020
938,2501796,artel,appliances.kitchen.oven,0.022036
1353,2501909,dauscher,appliances.kitchen.oven,0.021331
1346,3200550,dauscher,appliances.kitchen.meat_grinder,0.017683
4391,29100058,maxwell,appliances.personal.massager,0.016791
1394,3701387,dauscher,appliances.environment.vacuum,0.015864


In [7]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [113]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_base_line)

1.25495616539933

In [114]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_svd_preds)

1.065752985150627

## 구매 횟수 필터 있는 추천 시스템

In [8]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'appliances%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [9]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
31208,564055097,2700551,1
41604,559474168,3700851,1
41605,519303307,3700600,1
41606,523836574,3700621,1
41607,545495604,3700621,1
...,...,...,...
133,532485101,2400216,5
132,522804170,2501816,5
131,522567701,2701683,5
141,535781426,3701244,5


In [10]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [11]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
384989212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429474454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433754231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440756116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450756413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566221288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566227497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566235555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_final_limit = df_fin_limit.fillna(0)

In [13]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [14]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481341715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494312717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500246310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566094079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566122364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566144303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
matrix_limit = df_filtered_limit.values
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

In [16]:
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)

In [17]:
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [40]:
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 1000)

In [41]:
sigma_limit = np.diag(sigma_limit)

In [42]:
dot_matrix_limit = np.dot(U_limit, sigma_limit)

In [43]:
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

In [44]:
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [45]:
df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.000086,-0.000089,-0.000089,-0.000151,0.000011,-0.000089,-0.000268,-0.000087,-0.000165,-0.000185,...,0.000564,-0.000089,-0.000013,-0.000089,-0.000089,-0.000184,-0.000089,-0.000089,-0.000089,-0.000205
481341715,0.000045,-0.000030,-0.000030,-0.000025,-0.000012,-0.000030,0.000089,-0.000008,0.000079,-0.000038,...,-0.000220,-0.000030,-0.000002,-0.000030,-0.000030,-0.000071,-0.000030,-0.000030,-0.000030,-0.000050
494312717,0.000034,-0.000046,-0.000046,-0.000033,-0.000037,-0.000046,0.000465,-0.000205,0.000036,-0.000033,...,0.000183,-0.000046,0.000012,-0.000046,-0.000046,-0.000028,-0.000046,-0.000046,-0.000046,-0.000082
500246310,-0.000035,-0.000032,-0.000032,0.000026,-0.000101,-0.000032,0.000714,-0.000033,0.000063,-0.000026,...,0.000304,-0.000032,0.000011,-0.000032,-0.000032,-0.000019,-0.000032,-0.000032,-0.000032,0.000009
512363712,-0.000033,-0.000027,-0.000027,-0.000016,-0.000033,-0.000027,0.000474,0.000094,0.000055,0.000019,...,-0.000111,-0.000027,0.000004,-0.000027,-0.000027,0.000002,-0.000027,-0.000027,-0.000027,0.000022


In [46]:
already_rated, predictions = recommend_movies(df_svd_preds_limit, 512363712, product_data, df_ratings_limit, 10)

In [47]:
predictions

,product_id,brand,category_code,Predictions
3051,10900233,polaris,appliances.kitchen.mixer,0.033835
1344,10900341,dauscher,appliances.kitchen.mixer,0.030975
215,35700021,apollo,appliances.kitchen.kettle,0.026707
1728,3100618,lumme,appliances.kitchen.blender,0.024924
2517,3200540,redmond,appliances.kitchen.meat_grinder,0.022615
3445,6301922,elenberg,appliances.kitchen.kettle,0.021740
4170,11100266,unit,appliances.personal.scales,0.018473
2972,10900026,panasonic,appliances.kitchen.mixer,0.013993
107,29100011,gezatone,appliances.personal.massager,0.012742
3429,2900194,samsung,appliances.kitchen.microwave,0.011547


In [48]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.7565141480224759

In [49]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

0.49069946220778593